**BBM409 Machine Learning Lab. Assignment 3 Code and Report**

In this assignment, we were given a dataset of mails which were labeled as spam and ham mails and we were asked to create a naive bayes model which would classify the given mail. To achieve this we have used bag of words model to get the features as bigrams and unigrams using the CountVectorizer. 
After the classification part, we were asked to analyse the words that has more affection on the classification with use of TF-IDF and reimplement the naive bayes part, nextly, we were asked to find non-stopwords given the stopwords and lastly, we were asked to analyse the effect of the stopwords and calculate the performance metrics.

# Importing libraries

In [1]:
import re
from math import log

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [2]:
type(ENGLISH_STOP_WORDS)

frozenset

#Reading dataset

Reading and Handling the dataset, by removing the punctuations from the mails given.

In [3]:
dataset_df = pd.read_csv('emails.csv')

In [4]:
dataset_df.head()  # visualizing the head of the dataset

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


## Cleaning the dataset

In [5]:
dataset_df['text'] = dataset_df['text'].str.replace('\W', ' ')  # Removes punctuation
dataset_df['text'] = dataset_df['text'].str.lower()

# Part 1: Understanding the data

Let's pick three specific keywords that might be useful to understand the data: 

- please
- click
- easy

In our opinion, "**please**" will occur in *ham messages* more, because it's a kind word rather than a word that forces people. And since the keyword "**click**" is a word that forces people to do something, it will probably occur more in *spam messages*. For the keyword "**easy**", it may be occur more in *spam messages* because it is a word used to tell people about doing something. 

So let's have a look at their appearances in our dataset and see whether our guess' is true or not!

In [6]:
texts = dataset_df['text'].values
labels = dataset_df['spam'].values

In [7]:
def find_count_of_word(word, texts, labels) -> tuple:
    """Finds count of occurrence of a word in given texts.

        :param word: The word that is wanted to find its count
        :type word: str
        :param texts: Messages to search the word
        :type texts: numpy.ndarray
        :param labels: Labels of the messages
        :type labels: numpy.ndarray
        :return: A tuple that contains word's count in ham and spam messages, respectively
        """
    count_in_ham, count_in_spam = 0, 0

    for text, label in zip(texts, labels):
        if word in text:
            if label:
                count_in_spam += 1
            else:
                count_in_ham += 1

    return count_in_ham, count_in_spam

In [8]:
def find_prob_of_word(word, texts, labels) -> tuple:
    """Finds probability of occurrence of a word in given texts.

        :param word: The word that is wanted to find its probability
        :type word: str
        :param texts: Messages to search the word
        :type texts: numpy.ndarray
        :param labels: Labels of the messages
        :type labels: numpy.ndarray
        :return: A tuple that contains word's probability in ham and spam messages, respectively
        """
    count_in_ham, count_in_spam = 0, 0
    count_ham_words, count_spam_words = 0, 0

    for text, label in zip(texts, labels):
        count = text.split().count(word)
        if label:
            count_in_spam += count
            count_spam_words += len(text.split())
        else:
            count_in_ham += count
            count_ham_words += len(text.split())

    to_round = len(str(ham_count)) - 1
    return count_in_ham, count_in_spam, round(count_in_ham / count_ham_words, to_round), round(count_in_spam / count_spam_words, to_round)

In [9]:
# finding label counts to use them in probability calculations
ham_count = len(labels[dataset_df.spam == 0])  
spam_count = len(labels) - ham_count

In [10]:
for word in ['please', 'click', 'easy']:
    count_in_ham, count_in_spam = find_count_of_word(word, texts, labels)

    to_round = len(str(ham_count)) - 1
    prob_in_ham = round(count_in_ham / ham_count, to_round)
    prob_in_spam = round(count_in_spam / spam_count, to_round)
    print("#" * 25)
    print(f'for "{word}" keyword:')
    print('{0:-^35} | {1:-^35}'.format('Count in ham messages', 'Count in spam messages'))
    print(f'{count_in_ham:^35} | {count_in_spam:^35}')
    print()
    print('{0:-^35} | {1:-^35}'.format('Probability in ham messages', 'Probability in spam messages'))
    print(f'{prob_in_ham:^35} | {prob_in_spam:^35}')
print("#" * 25)

#########################
for "please" keyword:
-------Count in ham messages------- | ------Count in spam messages-------
               2475                 |                 369                

----Probability in ham messages---- | ---Probability in spam messages----
               0.568                |                0.27                
#########################
for "click" keyword:
-------Count in ham messages------- | ------Count in spam messages-------
                159                 |                 336                

----Probability in ham messages---- | ---Probability in spam messages----
               0.036                |                0.246               
#########################
for "easy" keyword:
-------Count in ham messages------- | ------Count in spam messages-------
                61                  |                 114                

----Probability in ham messages---- | ---Probability in spam messages----
               0.014                |     

In [11]:
for word in ['please', 'click', 'easy']:
    count_in_ham, count_in_spam, prob_in_ham, prob_in_spam = find_prob_of_word(word, texts, labels)

    print("#" * 25)
    print(f'for "{word}" keyword:')
    print('{0:-^35} | {1:-^35}'.format('Count in ham messages', 'Count in spam messages'))
    print(f'{count_in_ham:^35} | {count_in_spam:^35}')
    print()
    print('{0:-^35} | {1:-^35}'.format('Probability in ham messages', 'Probability in spam messages'))
    print(f'{prob_in_ham:^35} | {prob_in_spam:^35}')
print("#" * 25)

#########################
for "please" keyword:
-------Count in ham messages------- | ------Count in spam messages-------
               4386                 |                 581                

----Probability in ham messages---- | ---Probability in spam messages----
               0.004                |                0.002               
#########################
for "click" keyword:
-------Count in ham messages------- | ------Count in spam messages-------
                200                 |                 531                

----Probability in ham messages---- | ---Probability in spam messages----
                0.0                 |                0.002               
#########################
for "easy" keyword:
-------Count in ham messages------- | ------Count in spam messages-------
                62                  |                 154                

----Probability in ham messages---- | ---Probability in spam messages----
                0.0                 |     

## Statistics
||"please"|"click"|"easy"|
|------|------|------|------|
|Count in ham messages|2475|159|61|
|Count in spam messages|369|336|114|
|Probability in ham messages|0.568|0.036|0.014|
|Probability in spam messages|0.27|0.246|0.083|

From this table, we can see that the "**please**" keyword is being used in more than half of *ham messages*. On the other hand, the keyword "**click**" is being used more often in *spam messages* compared to *ham messages*.

And for the other keyword "**easy**" is not being used often in both of the message types. However, we can easily say that this word occurs in *spam messages* rather than *ham messages*.

Our predictions match up with the probabilities that we have found. Therefore, we can say this dataset is **feasible** for predicting whether a mail is ham or spam by using naïve bayes. And since our dataset is text, and naïve bayes uses laplace smoothing concept to find the probability of words, it should work well.

# Part 2: Implementing Naive Bayes

For this part, we have split the dataset and calculated some equations necessary for following parts.

Nextly, we have handled the unigram/bigram parts using countvectorizer, found the BoW of the given dataset, calculated the probabilities that are needed for naive bayes. Within this part we have used try except since some words absence were creating bias and handled the bias created by 0 value.

For prediction part, we have cleaned text first and calculated log probabilities to handle numerical underflow and took the sum of the probabilities.

Lastly, driver part calls the prediction part given the unigram/bigram parts done and calculates the performance overall for both parts.

## Splitting the dataset, train set and some calculations

In [12]:
train_set_df, test_set_df = train_test_split(dataset_df, test_size=0.2, random_state=4)

X_test = test_set_df['text'].values
y_test = test_set_df['spam'].values

# splitting train set as spam and ham
spam_mails_df = train_set_df[train_set_df['spam'] == 1]  
ham_mails_df = train_set_df[train_set_df['spam'] == 0]

# getting values as numpy arrays
X_train_spam = spam_mails_df['text'].values
X_train_ham = ham_mails_df['text'].values

# some variables to use later
len_train = len(train_set_df)
spam_count = len(X_train_spam)
ham_count = len(X_train_ham)
p_spam = spam_count / len_train
p_ham = ham_count / len_train

# laplace smoothing parameter
alpha = 1

## Unigram part

In [13]:
spam_vectorizer_unigram = CountVectorizer()
spam_bow_unigram = spam_vectorizer_unigram.fit_transform(X_train_spam)
spam_vocab_unigram = spam_vectorizer_unigram.get_feature_names_out()

ham_vectorizer_unigram = CountVectorizer()
ham_bow_unigram = ham_vectorizer_unigram.fit_transform(X_train_ham)
ham_vocab_unigram = ham_vectorizer_unigram.get_feature_names_out()

vocab_unigram = list(set(spam_vocab_unigram.tolist() + ham_vocab_unigram.tolist()))

n_spam_vocab_unigram = len(spam_vocab_unigram)
n_ham_vocab_unigram = len(ham_vocab_unigram)
n_vocab_unigram = len(vocab_unigram)

n_spam = spam_bow_unigram.sum()  # count of words in spam messages

n_ham = ham_bow_unigram.sum()  # count of words in ham messages

# initialization of words' probabilities
wordpbs_spam_unigram = {unique_word:0 for unique_word in vocab_unigram}
wordpbs_ham_unigram = {unique_word:0 for unique_word in vocab_unigram}

spam_sum = np.sum(spam_bow_unigram, axis=0)  # sum of all unique words' counts, separately
ham_sum = np.sum(ham_bow_unigram, axis=0)

# calculation of words' probabilities
for i in range(n_vocab_unigram):
    word = vocab_unigram[i]
    try:  # if there is a KeyError, it means the count of the given word is 0
        ind = spam_vectorizer_unigram.vocabulary_[word]  # to find the count of the word
        n_word_given_spam = spam_sum[0, ind]
    except KeyError:
        n_word_given_spam = 0
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha * n_vocab_unigram)
    wordpbs_spam_unigram[word] = p_word_given_spam

    try:  # same try-except logic in the above one
        ind = ham_vectorizer_unigram.vocabulary_[word]
        n_word_given_ham = ham_sum[0, ind]
    except KeyError:
        n_word_given_ham = 0
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha * n_vocab_unigram)
    wordpbs_ham_unigram[word] = p_word_given_ham

## Bigram part

In [14]:
spam_vectorizer = CountVectorizer(analyzer='word', ngram_range=(2, 2))
spam_bow = spam_vectorizer.fit_transform(X_train_spam)
spam_vocab_bigram = spam_vectorizer.get_feature_names_out()

ham_vectorizer = CountVectorizer(analyzer='word', ngram_range=(2, 2))
ham_bow = ham_vectorizer.fit_transform(X_train_ham)
ham_vocab_bigram = ham_vectorizer.get_feature_names_out()

vocab_bigram = list(set(spam_vocab_bigram.tolist() + ham_vocab_bigram.tolist()))

n_spam_vocab_bigram = len(spam_vocab_bigram)
n_ham_vocab_bigram = len(ham_vocab_bigram)
n_vocab_bigram = len(vocab_bigram)

n_spam = spam_bow.sum().sum()

n_ham = ham_bow.sum().sum()

# initialization of words' probabilities
wordpbs_spam_bigram = {unique_word:0 for unique_word in vocab_bigram}
wordpbs_ham_bigram = {unique_word:0 for unique_word in vocab_bigram}

spam_sum = np.sum(spam_bow, axis=0)
ham_sum = np.sum(ham_bow, axis=0)

# calculation of words' probabilities
for i in range(n_vocab_bigram):
    word = vocab_bigram[i]
    try:
        ind = spam_vectorizer.vocabulary_[word]
        n_word_given_spam = spam_sum[0, ind]
    except KeyError:
        n_word_given_spam = 0
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha * n_vocab_bigram)
    wordpbs_spam_bigram[word] = p_word_given_spam

    try:
        ind = ham_vectorizer.vocabulary_[word]
        n_word_given_ham = ham_sum[0, ind]
    except KeyError:
        n_word_given_ham = 0
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha * n_vocab_bigram)
    wordpbs_ham_bigram[word] = p_word_given_ham

## Prediction functions

In [15]:
def classify_unigram(message):
    # cleaning the given message
    message = re.sub('\W', ' ', message)
    message = message.lower().split()

    # initialization of probabilities
    p_spam_given_message = log(p_spam)
    p_ham_given_message = log(p_ham)
    
    # addition of log probabilities
    for word in message:
        if word in wordpbs_spam_unigram:
            p_spam_given_message += log(wordpbs_spam_unigram[word])
        if word in wordpbs_ham_unigram:
            p_ham_given_message += log(wordpbs_ham_unigram[word])

    if p_ham_given_message > p_spam_given_message:
        return 0
    elif p_spam_given_message > p_ham_given_message:
        return 1
    else:
        return None

In [16]:
def classify_bigram(message):
    message = re.sub('\W', ' ', message)
    message = message.lower().split()

    p_spam_given_message = log(p_spam)
    p_ham_given_message = log(p_ham)
    
    for i in range(len(message) - 1):
        word = ' '.join(message[i:i + 2])
        if word in wordpbs_spam_bigram:
            p_spam_given_message += log(wordpbs_spam_bigram[word])
        if word in wordpbs_ham_bigram:
            p_ham_given_message += log(wordpbs_ham_bigram[word])

    if p_ham_given_message > p_spam_given_message:
        return 0
    elif p_spam_given_message > p_ham_given_message:
        return 1
    else:
        return None

## Driver program and calculation of performance metrics

In [17]:
def calculate_performance(predictions, y_test):
    print(f'Accuracy: {accuracy_score(y_test, predictions)}')
    print(f'Precision: {precision_score(y_test, predictions)}')
    print(f'Recall: {recall_score(y_test, predictions)}')
    print(f'F1_score: {f1_score(y_test, predictions)}')

In [18]:
predictions_unigram = []
predictions_bigram = []

for text in X_test:
    predictions_unigram.append(classify_unigram(text))
    predictions_bigram.append(classify_bigram(text))

print('Unigram performance:')
calculate_performance(predictions_unigram, y_test)
print('\nBigram performance:')
calculate_performance(predictions_bigram, y_test)

Unigram performance:
Accuracy: 0.993891797556719
Precision: 0.9889705882352942
Recall: 0.9853479853479854
F1_score: 0.9871559633027523

Bigram performance:
Accuracy: 0.9790575916230366
Precision: 0.9844357976653697
Recall: 0.9267399267399268
F1_score: 0.9547169811320755


# Part 3

In this part we have used TF-IDF vectorizer to get the top and bottom words to find the words presence/absence most strongly affects the prediction model. After that we have implemented the part 2 for this new vocabulary we have found by, again calculating the probabilities for these and classification of them for both the unigram and the bigram model.

For the stopwords part we have, again, used TF-IDF vectorizer and found the non-stopwords with the use of the given stopwords list.
p.s. For this part, we did not include the word "subject" since it was not a word that was sent by the user.

Lastly for the analysis of the stopwords part, we have subtracted the given stopwords from our mails dataset and then created our model with these data just like the previous parts.

## Analyzing effect of the words on prediction

In [19]:
def extract_n_words_top(n, vocab, idfs):
    sorted_idfs_indices = np.argsort(idfs)

    words_idfs = np.empty((n - 1, 2), dtype=object)

    # since 'subject' is not a word written by sender, and every mail contains it, we skip it
    for i in range(1, len(sorted_idfs_indices)):
        if i >= n:
            break

        ind = sorted_idfs_indices[i]
        
        words_idfs[i - 1] = np.array([vocab[ind], idfs[ind]])
    
    return words_idfs

In [20]:
def extract_n_words_bottom(n, vocab, idfs):
    sorted_idfs_indices = np.argsort(idfs)[::-1]  # reverse sort to extract from bottom
    
    words_idfs = np.empty((n, 2), dtype=object)

    for i, ind in enumerate(sorted_idfs_indices):
        if i >= n:
            break
        
        words_idfs[i] = np.array([vocab[ind], idfs[ind]])
    
    return words_idfs

In [21]:
spam_tfidf_vectorizer = TfidfVectorizer(use_idf=True)
spam_tfidf_vectorizer.fit_transform(X_train_spam)
spam_vocab_tfidf = spam_tfidf_vectorizer.get_feature_names_out()
spam_idfs = spam_tfidf_vectorizer.idf_
spam_presence_words_idfs = extract_n_words_top(11, spam_vocab_tfidf, spam_idfs)
spam_absence_words_idfs = extract_n_words_bottom(10, spam_vocab_tfidf, spam_idfs)

ham_tfidf_vectorizer = TfidfVectorizer(use_idf=True)
ham_tfidf_vectorizer.fit_transform(X_train_ham)
ham_vocab_tfidf = ham_tfidf_vectorizer.get_feature_names_out()
ham_idfs = ham_tfidf_vectorizer.idf_
ham_presence_words_idfs = extract_n_words_top(11, ham_vocab_tfidf, ham_idfs)
ham_absence_words_idfs = extract_n_words_bottom(10, ham_vocab_tfidf, ham_idfs)

print("10 words whose presence most strongly predicts that the mail is ham:")
print(ham_presence_words_idfs)
print("\n10 words whose absence most strongly predicts that the mail is ham:")
print(ham_absence_words_idfs)
print("\n10 words whose presence most strongly predicts that the mail is spam:")
print(spam_presence_words_idfs)
print("\n10 words whose absence most strongly predicts that the mail is spam:")
print(spam_absence_words_idfs)

10 words whose presence most strongly predicts that the mail is ham:
[['to' '1.07098124935977']
 ['the' '1.0871156359690288']
 ['and' '1.1999064615693953']
 ['for' '1.222928551492548']
 ['you' '1.2240040131484557']
 ['of' '1.278170872731738']
 ['on' '1.3034789719310338']
 ['in' '1.3062011202190185']
 ['is' '1.358560731403664']
 ['have' '1.4392875740152884']]

10 words whose absence most strongly predicts that the mail is ham:
[['polach' '8.463936604468925']
 ['biting' '8.463936604468925']
 ['dostac' '8.463936604468925']
 ['practicalities' '8.463936604468925']
 ['bitter' '8.463936604468925']
 ['dosta' '8.463936604468925']
 ['dosage' '8.463936604468925']
 ['pracami' '8.463936604468925']
 ['ipcre' '8.463936604468925']
 ['biura' '8.463936604468925']]

10 words whose presence most strongly predicts that the mail is spam:
[['to' '1.1717932330051086']
 ['the' '1.2541861180235987']
 ['and' '1.326124499740307']
 ['you' '1.3311942190905577']
 ['your' '1.3452699473247423']
 ['of' '1.3543314980023

### Reimplementation of Part 2

In [22]:
specific_vocab_tfidf = list(set(spam_presence_words_idfs[:, 0].tolist() + spam_absence_words_idfs[:, 0].tolist() + ham_presence_words_idfs[:, 0].tolist() + ham_absence_words_idfs[:, 0].tolist()))

In [23]:
# initialization of words' probabilities
wordpbs_spam_tfidf = {unique_word:0 for unique_word in specific_vocab_tfidf}
wordpbs_ham_tfidf = {unique_word:0 for unique_word in specific_vocab_tfidf}

max_val = 9999999
# calculation of words' probabilities
for i in range(len(specific_vocab_tfidf)):
    word = specific_vocab_tfidf[i]
    try:  # if there is a KeyError, it means the count of the given word is 0
        ind = spam_tfidf_vectorizer.vocabulary_[word]  # to find the count of the word
        p_word_given_spam = 1 / spam_idfs[ind]
    except KeyError:
        p_word_given_spam = 1 / max_val

    wordpbs_spam_tfidf[word] = p_word_given_spam

    try:  # same try-except logic in the above one
        ind = ham_tfidf_vectorizer.vocabulary_[word]
        p_word_given_ham = 1 / ham_idfs[ind]
    except KeyError:
        p_word_given_ham = 1 / max_val

    wordpbs_ham_tfidf[word] = p_word_given_ham


def classify_unigram_tfidf(message):
    # cleaning the given message
    message = re.sub('\W', ' ', message)
    message = message.lower().split()

    # initialization of probabilities
    p_spam_given_message = log(p_spam)
    p_ham_given_message = log(p_ham)
    
    # addition of log probabilities
    for word in message:
        if word in wordpbs_spam_tfidf:
            p_spam_given_message += log(wordpbs_spam_tfidf[word])
        if word in wordpbs_ham_tfidf:
            p_ham_given_message += log(wordpbs_ham_tfidf[word])

    if p_ham_given_message > p_spam_given_message:
        return 0
    elif p_spam_given_message > p_ham_given_message:
        return 1
    else:
        return None

In [24]:
predictions = []

for text in X_test:
    predictions.append(classify_unigram_tfidf(text))

print('Reimplemented bayes performance:')
calculate_performance(predictions, y_test)
print('\n', classification_report(y_test, predictions))

Reimplemented bayes performance:
Accuracy: 0.7626527050610821
Precision: 1.0
Recall: 0.003663003663003663
F1_score: 0.0072992700729927005

               precision    recall  f1-score   support

           0       0.76      1.00      0.87       873
           1       1.00      0.00      0.01       273

    accuracy                           0.76      1146
   macro avg       0.88      0.50      0.44      1146
weighted avg       0.82      0.76      0.66      1146



In [25]:
spam_tfidf_vectorizer_bigram = TfidfVectorizer(use_idf=True, ngram_range=(2, 2))
spam_tfidf_vectorizer_bigram.fit_transform(X_train_spam)
spam_vocab_tfidf_bigram = spam_tfidf_vectorizer_bigram.get_feature_names_out()
spam_idfs_bigram = spam_tfidf_vectorizer_bigram.idf_
spam_presence_words_idfs_bigram = extract_n_words_top(11, spam_vocab_tfidf_bigram, spam_idfs_bigram)
spam_absence_words_idfs_bigram = extract_n_words_bottom(10, spam_vocab_tfidf_bigram, spam_idfs_bigram)

ham_tfidf_vectorizer_bigram = TfidfVectorizer(use_idf=True, ngram_range=(2, 2))
ham_tfidf_vectorizer_bigram.fit_transform(X_train_ham)
ham_vocab_tfidf_bigram = ham_tfidf_vectorizer_bigram.get_feature_names_out()
ham_idfs_bigram = ham_tfidf_vectorizer_bigram.idf_
ham_presence_words_idfs_bigram = extract_n_words_top(11, ham_vocab_tfidf_bigram, ham_idfs_bigram)
ham_absence_words_idfs_bigram = extract_n_words_bottom(10, ham_vocab_tfidf_bigram, ham_idfs_bigram)

specific_vocab_tfidf_bigram = list(set(spam_presence_words_idfs_bigram[:, 0].tolist() + spam_absence_words_idfs_bigram[:, 0].tolist() + ham_presence_words_idfs_bigram[:, 0].tolist() + ham_absence_words_idfs_bigram[:, 0].tolist()))

# initialization of words' probabilities
wordpbs_spam_tfidf_bigram = {unique_word:0 for unique_word in specific_vocab_tfidf_bigram}
wordpbs_ham_tfidf_bigram = {unique_word:0 for unique_word in specific_vocab_tfidf_bigram}

max_val = 9999999
# calculation of words' probabilities
for i in range(len(specific_vocab_tfidf_bigram)):
    word = specific_vocab_tfidf_bigram[i]
    try:  # if there is a KeyError, it means the count of the given word is 0
        ind = spam_tfidf_vectorizer_bigram.vocabulary_[word]  # to find the count of the word
        p_word_given_spam = 1 / spam_idfs_bigram[ind]
    except KeyError:
        p_word_given_spam = 1 / max_val

    wordpbs_spam_tfidf_bigram[word] = p_word_given_spam

    try:  # same try-except logic in the above one
        ind = ham_tfidf_vectorizer_bigram.vocabulary_[word]
        p_word_given_ham = 1 / ham_idfs_bigram[ind]
    except KeyError:
        p_word_given_ham = 1 / max_val

    wordpbs_ham_tfidf_bigram[word] = p_word_given_ham


def classify_bigram_tfidf(message):
    # cleaning the given message
    message = re.sub('\W', ' ', message)
    message = message.lower().split()

    # initialization of probabilities
    p_spam_given_message = log(p_spam)
    p_ham_given_message = log(p_ham)
    
    # addition of log probabilities
    for i in range(len(message) - 1):
        word = ' '.join(message[i:i + 2])
        if word in wordpbs_spam_tfidf_bigram:
            p_spam_given_message += log(wordpbs_spam_tfidf_bigram[word])
        if word in wordpbs_ham_tfidf_bigram:
            p_ham_given_message += log(wordpbs_ham_tfidf_bigram[word])

    if p_ham_given_message > p_spam_given_message:
        return 0
    elif p_spam_given_message > p_ham_given_message:
        return 1
    else:
        return None

In [26]:
predictions = []

for text in X_test:
    predictions.append(classify_bigram_tfidf(text))

print('Reimplemented bayes performance:')
calculate_performance(predictions, y_test)
print('\n', classification_report(y_test, predictions))

Reimplemented bayes performance:
Accuracy: 0.7696335078534031
Precision: 0.8461538461538461
Recall: 0.040293040293040296
F1_score: 0.07692307692307693

               precision    recall  f1-score   support

           0       0.77      1.00      0.87       873
           1       0.85      0.04      0.08       273

    accuracy                           0.77      1146
   macro avg       0.81      0.52      0.47      1146
weighted avg       0.79      0.77      0.68      1146



## Stopwords

In [27]:
def extract_n_words_top(n, vocab, idfs, stop_words):
    sorted_idfs_indices = np.argsort(idfs)

    words_idfs = np.empty((n - 1, 2), dtype=object)
    
    count = 0
    # since 'subject' is not a word written by sender, and every mail contains it, we skip it
    for i in range(1, len(sorted_idfs_indices)):
        if count > 9:
            break

        ind = sorted_idfs_indices[i]
        
        word = vocab[ind]

        if word not in stop_words:
            words_idfs[count] = np.array([word, idfs[ind]])
            count += 1
    
    return words_idfs

In [28]:
stop_words = list(ENGLISH_STOP_WORDS)

spam_tfidf_vectorizer = TfidfVectorizer(use_idf=True)
spam_tfidf_vectorizer.fit_transform(X_train_spam)
spam_vocab_tfidf = spam_tfidf_vectorizer.get_feature_names_out()
spam_idfs = spam_tfidf_vectorizer.idf_
spam_presence_words_idfs = extract_n_words_top(11, spam_vocab_tfidf, spam_idfs, stop_words)

ham_tfidf_vectorizer = TfidfVectorizer(use_idf=True)
ham_tfidf_vectorizer.fit_transform(X_train_ham)
ham_vocab_tfidf = ham_tfidf_vectorizer.get_feature_names_out()
ham_idfs = ham_tfidf_vectorizer.idf_
ham_presence_words_idfs = extract_n_words_top(11, ham_vocab_tfidf, ham_idfs, stop_words)

In [29]:
print('10 non-stopwords that most strongly predict that the mail is spam:')
print(spam_presence_words_idfs)
print('\n10 non-stopwords that most strongly predict that the mail is ham:')
print(ham_presence_words_idfs)

10 non-stopwords that most strongly predict that the mail is spam:
[['com' '2.113318436057805']
 ['http' '2.2468498286823277']
 ['information' '2.3826513698413896']
 ['just' '2.389950672323001']
 ['click' '2.3973036466282602']
 ['business' '2.4464628825863435']
 ['email' '2.490034130879984']
 ['best' '2.490034130879984']
 ['new' '2.506361024167412']
 ['time' '2.5104847413512745']]

10 non-stopwords that most strongly predict that the mail is ham:
[['vince' '1.4432991684301153']
 ['enron' '1.5310010269564895']
 ['cc' '1.6920010486293227']
 ['kaminski' '1.8040018645134381']
 ['thanks' '1.8888607638693047']
 ['2000' '1.9193055931054273']
 ['pm' '1.9447893165285293']
 ['ect' '1.995461782089683']
 ['know' '2.0382575359667925']
 ['hou' '2.0578820414757777']]


## Analyzing effect of the stopwords

**Why might it make sense to remove stop words when interpreting the model?**

While using bag of words based methods, such as, countVectorizer or tfidf that works on counts and frequency of the words, removal of the stopwords helps since it lowers the dimensional space and also a few stop words won't affect the analysis. Removing stopwords also helps the model to get the affective words rather than focusing on commonly used words.

**Why might it make sense to keep stop words?**

With the removal of the stopwords, context of the given texts might get omitted, which means that the removal will damage the model by changing the meanings of the text. For example, if a negation word have got removed from a sentence, the context of the sentence will have changed the opposite way. So, while removing the stopwords, we have to be cautious of what is being dropped.


In [30]:
stop_words = list(ENGLISH_STOP_WORDS)

In [31]:
for index,item in enumerate(texts):
  words = [word for word in item.split() if word.lower() not in stop_words]
  new_sentence = " ".join(words)
  texts[index] = new_sentence

In [32]:
dataset_df.head()

,text,spam
0,subject naturally irresistible corporate ident...,1
1,subject stock trading gunslinger fanny merrill...,1
2,subject unbelievable new homes easy im wanting...,1
3,subject 4 color printing special request addit...,1
4,subject money software cds software compatibil...,1


In [33]:
train_set_df, test_set_df = train_test_split(dataset_df, test_size=0.2, random_state=4)

X_test = test_set_df['text'].values
y_test = test_set_df['spam'].values

# splitting train set as spam and ham
spam_mails_df = train_set_df[train_set_df['spam'] == 1]  
ham_mails_df = train_set_df[train_set_df['spam'] == 0]

# getting values as numpy arrays
X_train_spam = spam_mails_df['text'].values
X_train_ham = ham_mails_df['text'].values

# some variables to use later
len_train = len(train_set_df)
spam_count = len(X_train_spam)
ham_count = len(X_train_ham)
p_spam = spam_count / len_train
p_ham = ham_count / len_train

# laplace smoothing parameter
alpha = 1

Unigram - no stopwords

In [34]:
spam_vectorizer_unigram = CountVectorizer()
spam_bow_unigram = spam_vectorizer_unigram.fit_transform(X_train_spam)
spam_vocab_unigram = spam_vectorizer_unigram.get_feature_names_out()

ham_vectorizer_unigram = CountVectorizer()
ham_bow_unigram = ham_vectorizer_unigram.fit_transform(X_train_ham)
ham_vocab_unigram = ham_vectorizer_unigram.get_feature_names_out()

vocab_unigram = list(set(spam_vocab_unigram.tolist() + ham_vocab_unigram.tolist()))

n_spam_vocab_unigram = len(spam_vocab_unigram)
n_ham_vocab_unigram = len(ham_vocab_unigram)
n_vocab_unigram = len(vocab_unigram)

n_spam = spam_bow_unigram.sum()  # count of words in spam messages

n_ham = ham_bow_unigram.sum()  # count of words in ham messages

# initialization of words' probabilities
wordpbs_spam_unigram = {unique_word:0 for unique_word in vocab_unigram}
wordpbs_ham_unigram = {unique_word:0 for unique_word in vocab_unigram}

spam_sum = np.sum(spam_bow_unigram, axis=0)  # sum of all unique words' counts, separately
ham_sum = np.sum(ham_bow_unigram, axis=0)

# calculation of words' probabilities
for i in range(n_vocab_unigram):
    word = vocab_unigram[i]
    try:  # if there is a KeyError, it means the count of the given word is 0
        ind = spam_vectorizer_unigram.vocabulary_[word]  # to find the count of the word
        n_word_given_spam = spam_sum[0, ind]
    except KeyError:
        n_word_given_spam = 0
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha * n_vocab_unigram)
    wordpbs_spam_unigram[word] = p_word_given_spam

    try:  # same try-except logic in the above one
        ind = ham_vectorizer_unigram.vocabulary_[word]
        n_word_given_ham = ham_sum[0, ind]
    except KeyError:
        n_word_given_ham = 0
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha * n_vocab_unigram)
    wordpbs_ham_unigram[word] = p_word_given_ham

Bigram - no stopword

In [35]:
spam_vectorizer = CountVectorizer(analyzer='word', ngram_range=(2, 2))
spam_bow = spam_vectorizer.fit_transform(X_train_spam)
spam_vocab_bigram = spam_vectorizer.get_feature_names_out()

ham_vectorizer = CountVectorizer(analyzer='word', ngram_range=(2, 2))
ham_bow = ham_vectorizer.fit_transform(X_train_ham)
ham_vocab_bigram = ham_vectorizer.get_feature_names_out()

vocab_bigram = list(set(spam_vocab_bigram.tolist() + ham_vocab_bigram.tolist()))

n_spam_vocab_bigram = len(spam_vocab_bigram)
n_ham_vocab_bigram = len(ham_vocab_bigram)
n_vocab_bigram = len(vocab_bigram)

n_spam = spam_bow.sum().sum()

n_ham = ham_bow.sum().sum()

# initialization of words' probabilities
wordpbs_spam_bigram = {unique_word:0 for unique_word in vocab_bigram}
wordpbs_ham_bigram = {unique_word:0 for unique_word in vocab_bigram}

spam_sum = np.sum(spam_bow, axis=0)
ham_sum = np.sum(ham_bow, axis=0)

# calculation of words' probabilities
for i in range(n_vocab_bigram):
    word = vocab_bigram[i]
    try:
        ind = spam_vectorizer.vocabulary_[word]
        n_word_given_spam = spam_sum[0, ind]
    except KeyError:
        n_word_given_spam = 0
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha * n_vocab_bigram)
    wordpbs_spam_bigram[word] = p_word_given_spam

    try:
        ind = ham_vectorizer.vocabulary_[word]
        n_word_given_ham = ham_sum[0, ind]
    except KeyError:
        n_word_given_ham = 0
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha * n_vocab_bigram)
    wordpbs_ham_bigram[word] = p_word_given_ham

In [36]:
predictions_unigram = []
predictions_bigram = []

for text in X_test:
    predictions_unigram.append(classify_unigram(text))
    predictions_bigram.append(classify_bigram(text))

print('Unigram performance:')
calculate_performance(predictions_unigram, y_test)
print('\nBigram performance:')
calculate_performance(predictions_bigram, y_test)

Unigram performance:
Accuracy: 0.9947643979057592
Precision: 0.992619926199262
Recall: 0.9853479853479854
F1_score: 0.988970588235294

Bigram performance:
Accuracy: 0.9790575916230366
Precision: 0.973384030418251
Recall: 0.9377289377289377
F1_score: 0.9552238805970149


Since we are using countVectorizer, removing the stopwords helps, because it lowers the dimensional space and also the results of the program has not changed that much with this action taken.

# Part 4: Calculation and Analysis of Performance Metrics

||Accuracy|Precision|Recall|F1Score|
|------|------|------|------|------|
|Unigram|0.994|0.989|0.985|0.987|
|Bigram|0.979|0.984|0.927|0.955|
|Unigram-wo/stop|0.994|0.992|0.985|0.988|
|Bigram-wo/stop|0.979|0.973|0.937|0.955|